# Parse and Visualize Model Outputs

## Codex Predictions

In [1]:
codex_lines = open('../outputs/gsm8k/train_codex_complex.txt').readlines()

In [2]:
len(codex_lines)

1036165

In [3]:
codex_lines[0]

'Question 0: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\n'

In [4]:
codex_lines[1]

'Answer: Natalia sold 48/2 = <<48/2=24>>24 clips in May.\n'

In [24]:
codex_lines[2]

'Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n'

In [25]:
codex_lines[3]

'#### 72\n'

In [26]:
codex_lines[4]

'Model output 0: \n'

In [27]:
codex_lines[5]

'If Natalia sold 48 clips in April, she sold 48/2 = 24 clips in May.\n'

In [28]:
codex_lines[6]

'In total, Natalia sold 48 + 24 = 72 clips in April and May.\n'

In [29]:
codex_lines[7]

'The answer is 72\n'

In [30]:
codex_lines[8]

'Per-step decode: <<\' step\'>> 1.0000 <<\'!\'>> 0.0000 <<\' stepped\'>> 0.0000 <<\' stepping\'>> 0.0000 <<\' step\'>> 1.0000 <<\' steps\'>> 0.0000 ||| <<\'\\n\'>> 0.9862 <<\'!\'>> 0.0002 <<\'.\'>> 0.0090 <<\'\\n\'>> 0.9862 <<\' \'>> 0.0010 <<\':\'>> 0.0025 ||| <<\'If\'>> 0.1009 <<\'If\'>> 0.1009 <<\'She\'>> 0.0416 <<\'The\'>> 0.0394 <<\'Nat\'>> 0.3286 <<\'In\'>> 0.2685 ||| <<\' Nat\'>> 0.6018 <<\' Nat\'>> 0.6018 <<\' in\'>> 0.0261 <<\' we\'>> 0.0125 <<\' 48\'>> 0.0335 <<\' she\'>> 0.2588 ||| <<\'alia\'>> 0.9968 <<\'ale\'>> 0.0001 <<\'ilia\'>> 0.0001 <<\'lia\'>> 0.0002 <<\' sold\'>> 0.0013 <<\'alia\'>> 0.9968 ||| <<\' sold\'>> 0.9573 <<\' sells\'>> 0.0217 <<\' sold\'>> 0.9573 <<\' is\'>> 0.0022 <<"\'s">> 0.0028 <<\' had\'>> 0.0024 ||| <<\' 48\'>> 0.4455 <<\' clips\'>> 0.3353 <<\' half\'>> 0.1318 <<\' to\'>> 0.0351 <<\' 48\'>> 0.4455 <<\' twice\'>> 0.0058 ||| <<\' clips\'>> 0.9248 <<\' clips\'>> 0.9248 <<\' friends\'>> 0.0062 <<\' to\'>> 0.0058 <<\' of\'>> 0.0109 <<\' in\'>> 0.0345 ||| 

In [31]:
codex_lines[9]

'Model output 1: \n'

In [4]:
def parse_codex_outputs(lines):
    questions = []
    answers = []
    ans_pred = []
    per_step_prob = []
    mode = ""
    ans_list = None
    for li, l in enumerate(lines):
        if(l.startswith('Question')):
            if(mode == 'm' or mode == 'none'):
                mode = 'none'
                continue
            q = l
            mode = 'q'
            if(ans_list is not None):
                ans_pred.append(ans_list)
                per_step_prob.append(prob_list)
            ans_list = []
            prob_list = []
        elif(l.startswith('Answer: ')):
            questions.append(q)
            a = [l]
            mode = 'a'
        elif(l.startswith('Model output')):
            if(l.startswith('Model output 0')):
                answers.append(''.join(a))
            mode = 'm'
            m = [l]
        elif(l.startswith('Per-step')):
            ans_list.append(''.join(m))
            mode = 'p'
            prob_list.append(l)
        else:
            if(mode == 'a'):
                a.append(l)
            elif(mode == 'm'):
                m.append(l)
            elif(mode == 'p'): 
                pass
            elif(mode == 'none'):
                pass
            else:
                print(mode)
                print(li)
                print(lines[li - 1])
                print(l)
                raise ValueError() 
    ans_pred.append(ans_list)
    per_step_prob.append(prob_list)
    
    per_step_prob_ = []
    for p in per_step_prob:
        p_ = []
        for pi in p: 
            p_.append(pi.split(' ||| '))
        per_step_prob_.append(p_)
    return questions, answers, ans_pred, per_step_prob_

In [5]:
questions, answers, ans_pred, per_step_prob = parse_codex_outputs(codex_lines)

In [6]:
len(questions)

2489

In [7]:
len(answers)

2489

In [8]:
len(ans_pred[-1])

50

In [12]:
per_step_prob[-1][0][0]

"Per-step decode: <<' step'>> 0.9999 <<'!'>> 0.0000 <<' stepped'>> 0.0000 <<' stepping'>> 0.0000 <<' step'>> 0.9999 <<' steps'>> 0.0001"